In [1]:
# numpy provides import array and linear algebra utilities
import numpy as np
# casadi is a library for symbolic computation
import casadi as ca
# the transformations module provides functions for creating transformations
import utils.transformations as ta
# some libraries used for visualizations
from example_robot_data import load
from pinocchio.visualize import MeshcatVisualizer
import time
import meshcat

# Robotics Course : Control Exercise

## Part 3 : Constraint-based robot task specification

The two previous parts focussed on control (both in joint space and in task space).
This part shifts the focus to task specification, i.e., telling the robot what it should do.
Specifically, we will investigate *constraint-based task specification*, where the desired behaviour of the robot is specified in terms of constraints (task functions).
Then, a feedback controller (similar to the controllers designed in part 2), is used to regulate the task functions to their desired values.

A simulation environment and controller is provided, and your objective is to specify the task functions to accomplish the desired behaviour. 

### Load a robot and create a visualization environment

In [2]:
robot = load("panda")
viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)

# Add floor
material_black = meshcat.geometry.MeshPhongMaterial()
material_black.color = int(100) * 256**2 + int(100) * 256 + int(100)
viz.viewer['floor'].set_object(
    meshcat.geometry.Box([1.25, 1, 0.01]), material_black)
viz.viewer['floor'].set_transform(
    ta.SE3_from_xyz_rpy([0.375, 0, -0.01], [0, 0, 0]))

# Set initial joint angles
q0 = np.array([-0.37, -0.88, -0.24, -2.35, -0.18,  1.47, 0.24])
viz.display(np.append(q0, [0, 0]))
# viz.viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


### Define CasADi functions for robot and the environment

In [3]:
# load casadi function for the forward kinematics of the end effector
# this function was generate using Pinocchio as shown in exercise 1
T_world_ee_fn = ca.Function.load('panda_T_world_ee.casadi')

# define a symbolic variable for time
t_sym = ca.MX.sym("t_sym")

# define the target pose
T_world_goal = ta.SE3_from_xyz_rpy(
    xyz=[0.5-0.2*ca.cos(t_sym), 0, 0.5], rpy=[np.pi, 0, 0])

# convert to a casadi function to evaluate numerically (or symbolically)
T_world_goal_fn = ca.Function("T_world_goal_fn", [t_sym], [
                              T_world_goal], ["t"], ["T_world_goal"])

# Define obstacle trajectory
T_world_obstacle = ta.SE3_from_xyz_rpy(
    xyz=[0.5, -0.3*ca.cos(1*t_sym), 0.5], rpy=[0, 0, 0])
# convert to a casadi function to evaluate numerically (or symbolically)
T_world_obstacle_fn = ca.Function("T_world_obstacle_fn", [t_sym], [
                                  T_world_obstacle], ["t"], ["T_world_obstacle"])

In [4]:
# Add frames
viz.viewer['T_w_ee'].set_object(meshcat.geometry.triad(0.1))
viz.viewer['T_w_ee'].set_transform(T_world_ee_fn(q0).full())

# add a target frame vizualization
viz.viewer['target'].set_object(meshcat.geometry.triad(0.1))
viz.viewer['target'].set_transform(T_world_goal_fn(0).full())

# Add obstacle
material_obstacle = meshcat.geometry.MeshPhongMaterial()
material_obstacle.color = int(100) * 256**2 + int(100) * 256 + int(100)
viz.viewer['obstacle'].set_object(
    meshcat.geometry.Sphere(0.1), material_obstacle)
viz.viewer['obstacle'].set_transform(T_world_obstacle_fn(0).full())

### The simulation loop


In [6]:
def simulate(controller, q0):

    # set parameters for the simulation
    control_freq = 100  # hz
    render_freq = 32  # hz
    sim_time = 10  # sec

    q = q0

    for t in np.arange(0, sim_time, 1/control_freq):

        # compute the joint velocities using the robot controller function
        dq = controller(q, t)

        # update the joint positions
        # here, modeled as a perfect simulator.
        q = q + dq * (1/control_freq)

        # update the positions of the frames and obstacles
        viz.viewer['target'].set_transform(T_world_goal_fn(t).full())
        viz.viewer['T_w_ee'].set_transform(T_world_ee_fn(q).full())
        viz.viewer['obstacle'].set_transform(T_world_obstacle_fn(t).full())

        # update the vizualization at the correct rate
        if t % (1/render_freq) < 1/control_freq:
            # add two zeroes to end of q for gripper joints
            q_robot = np.append(q, [0, 0])
            viz.display(q_robot)
            time.sleep(1 / (render_freq))

### Constraint-based robot task specification

With the approach, the idea is to specify task functions that describe the desired behavior of the robot.
Specifically, the following notation is used:
$$
e(q,t) \xrightarrow[w]{K} e_\text{des}(t)
$$
where
- $e$ is the task function, consisting of the robot joints, $q$, and time, $t$.
- $e_\text{des}$ is the desired value of the task function.
- $K^{-1}$ is the time constant with which the error should decay.
- $w$ is a weight to indicate the importance relative to other task functions.

Furthermore, it is also possible to model inequality constraint, expressed as
$$
h(q,t) \xrightarrow[w]{K, \geq} h_\text{lower}(t)
$$
- $h$ is the inequality constraint, consisting of the robot joints, $q$, and time, $t$.
- $h_\text{lower}$ is the lower bound of the inequality constraint.
- $K^{-1}$ is the time constant at which the controller can approach the bound of the constraint.
- $w$ is a weight to indicate the importance relative to other task functions and inequality constraints.

#### Controller

Provided below is a slightly generalized version of the controller investigated in part 2. 
Specifically, at each time step the following QP is solved:
$$
\begin{align*} 
        \min_{\dot{q}, \epsilon} \quad
                & \epsilon ^T W \epsilon + \mu \dot{q}^T R \dot{q}
        \\
        \textrm{s.t.} \quad
                & \frac{\partial \tilde{e} (q, t)}{\partial q}\dot{q}  = - K \tilde{e}(q, t)- \frac{\partial \tilde{e} (q, t)}{\partial t} + \epsilon_e \\
                & \frac{\partial \tilde{h} (q, t)}{\partial q}\dot{q}  \leq - K \tilde{h}(q, t)- \frac{\partial \tilde{h} (q, t)}{\partial t} + \epsilon_h
\end{align*}
$$
where $\epsilon = [\epsilon_e^T, \epsilon_h^T]^T$, $\tilde{e} = e(q,t)- e_\text{des}(t)$, and $\tilde{h} = h(q,t)- h_\text{lower}(t)$


In [7]:
class ConstraintBasedController:
    """
    A constraint-based feedback controller for robotic systems.
    """

    def __init__(self, num_joints, R, mu=1e-5):
        """
        Initializes the constraint-based feedback controller.

        Parameters:
        num_joints (int): Number of robot joints.
        R (numpy.ndarray): weight of each joint
        mu (float, optional): Regularization coefficient. Default is 1e-5.
        """
        self.nq = num_joints  # Number of robot joints
        self.opti_QP = ca.Opti('conic')  # Create an optimization problem

        # Define parameters for joint states and time
        self.q = self.opti_QP.parameter(self.nq)
        self.t = self.opti_QP.parameter()

        # Define decision variables (joint velocities)
        self.u = self.opti_QP.variable(self.nq)

        # Define the objective function with regularization
        self.objective = mu * self.u.T @ np.diag(R) @ self.u
        self.initialized = False

    def add_task(self, expr, target, gain_k, hard=False, weight=1):
        """
        Adds a task to the controller.

        Parameters:
        expr (casadi.MX): Expression representing the task.
        target (casadi.MX): Target value for the task.
        gain_k (float): Gain for the task.
        hard (bool, optional): If True, the task is a hard constraint. Default is False.
        weight (float, optional): Weight of the task in the objective function. Default is 1.

        Returns:
        None
        """
        # Compute the error and its derivative
        e = expr - target
        error_dot = ca.jtimes(e, self.q, self.u) + ca.jacobian(e, self.t)

        if hard:
            # Add a hard constraint
            self.opti_QP.subject_to(error_dot == -gain_k * e)
        else:
            # Add a soft constraint to the objective function
            eps = error_dot + gain_k * e
            self.objective += weight * (eps.T @ eps)

    def add_inequality(self, expr, gain_k, ub=None, lb=None, hard=False, weight=1):
        """
        Adds an inequality constraint to the controller.

        Parameters:
        expr (casadi.MX): Expression representing the inequality constraint.
        gain_k (float): Gain for the constraint.
        ub (float, optional): Upper bound of the inequality constraint. Default is None.
        lb (float, optional): Lower bound of the inequality constraint. Default is None.
        hard (bool, optional): If True, the constraint is hard. Default is False.
        weight (float, optional): Weight of the constraint in the optimization problem. Default is 1.

        Returns:
        None
        """
        # Define slack variable for soft constraints
        eps = 0 if hard else self.opti_QP.variable(expr.shape[0])
        if not hard:
            # Add the slack variable to the objective function
            self.objective += weight * (eps.T @ eps)

        if ub is not None:
            # Compute the error and its derivative for the upper bound
            e = expr - ub
            error_dot = ca.jtimes(e, self.q, self.u) + ca.jacobian(e, self.t)
            # jtimes is a more efficient way of computing: ca.jacobian(e, self.q)@self.u

            # Add an upper bound constraint
            self.opti_QP.subject_to(error_dot <= -gain_k * e + eps)

        if lb is not None:
            # Compute the error and its derivative for the lower bound
            e = expr - lb
            error_dot = ca.jtimes(e, self.q, self.u) + ca.jacobian(e, self.t)
            # Add a lower bound constraint
            self.opti_QP.subject_to(error_dot >= -gain_k * e + eps)

    def initialize(self):
        """
        Initializes the controller by setting up the optimization problem.

        Returns:
        None
        """
        # Set the objective function of the QP problem
        self.opti_QP.minimize(self.objective)

        # Set solver options and initialize the solver
        p_opts = {"expand": True, "printLevel": "none"}
        s_opts = {}
        self.opti_QP.solver('qpoases', p_opts, s_opts)

        # Convert the optimization problem into a callable function
        self.controller_func = self.opti_QP.to_function(
            'solver', [self.q, self.t], [self.u], ['q', 't'], ['dq'])
        self.initialized = True

    def __call__(self, q, t):
        """
        Executes the controller with the given state and time.

        Parameters:
        q (numpy.ndarray): State vector.
        t (float): Current time.

        Returns:
        numpy.ndarray: Desired joint velocities if initialized, otherwise zeros.
        """
        if self.initialized:
            # Call the controller function with the current state and time
            return self.controller_func(q, t).full().ravel()
        else:
            print("First initialize the controller before calling it.")
            return np.zeros(self.nq)

### Task 3.1 : Add collision avoidance

- 3.1.1 Modify the task specification:
    - The end-effector remains at least 0.2 m from the origin of the obstacle, 
    - While following the desired trajectory.
- 3.1.2 What are some inherent limitations of the collision avoidance behavior? How would you improve it?

In [21]:
controller = ConstraintBasedController(num_joints=7, mu=1e-4, R=np.ones(7))

q = controller.q
t = controller.t

# get the relevant transformations
T_world_ee = T_world_ee_fn(q)
T_world_goal = T_world_goal_fn(t)

########################
# track the trajectory #
########################
T_ee_goal = ta.inverse_SE3(T_world_ee) @ T_world_goal

controller.add_task(
    expr = T_ee_goal[:3, 3],
    target = 0,
    gain_k = 5,
    weight = 1
)

R_ee_goal = T_ee_goal[:3, :3]
controller.add_task(
    expr=ta.axis_angle_from_rotation_matrix(R_ee_goal),
    target=0,
    gain_k=5,
    weight=1
)

####################
# add joint limits #
####################

controller.add_inequality(
    expr = q,
    ub=robot.model.upperPositionLimit[:7],
    lb=robot.model.lowerPositionLimit[:7],
    gain_k = 5,
    hard = True
)

###########################
# add collision avoidance #
###########################

T_world_obstacle = T_world_obstacle_fn(t)

##################################
# TODO: define constraints

# Compute the distance between the end effector and the obstacle
distance = ca.norm_2(T_world_obstacle[:3, 3] - T_world_ee[:3, 3])

# Add a constraint to keep the distance between the end effector and the obstacle above a certain threshold
controller.add_inequality(
    expr=distance,
    lb=0.2,
    gain_k=5,
    hard=True,
    weight=1e4
)




##################################


#############################
# initialize the controller #
#############################
controller.initialize()

simulate(controller, q0)


qpOASES -- An Implementation of the Online Active Set Strategy.
Copyright (C) 2007-2015 by Hans Joachim Ferreau, Andreas Potschka,
Christian Kirches et al. All rights reserved.

qpOASES is distributed under the terms of the 
GNU Lesser General Public License 2.1 in the hope that it will be 
useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
See the GNU Lesser General Public License for more details.


qpOASES -- An Implementation of the Online Active Set Strategy.
Copyright (C) 2007-2015 by Hans Joachim Ferreau, Andreas Potschka,
Christian Kirches et al. All rights reserved.

qpOASES is distributed under the terms of the 
GNU Lesser General Public License 2.1 in the hope that it will be 
useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
See the GNU Lesser General Public License for more details.



### Task 3.2 : Remain fixed distance and perpendicular to the sphere

- 3.2.1 Modify the task specification:
    - Remain 0.15 m from the origin of the sphere (previously the obstacle)
    - Keep the z-axis of the end effector (shown in blue) perpendicular to the sphere
    - Change it to keep the x-axis or y-axis perpendicular to sphere

In [20]:
controller = ConstraintBasedController(num_joints=7, mu=1e-4, R=np.ones(7))

q = controller.q
t = controller.t

# get the relevant transformations
T_world_ee = T_world_ee_fn(q)
T_world_obstacle = T_world_obstacle_fn(t)
T_world_goal = T_world_goal_fn(t)

####################
# add joint limits #
####################

controller.add_inequality(
    expr=q,
    ub=robot.model.upperPositionLimit[:7],
    lb=robot.model.lowerPositionLimit[:7],
    gain_k=10,
    hard=True
)

###########################################################
# maintain desired position and perpendicular orientation #
###########################################################

##################################
# TODO: define constraints

T_ee_obstacle = ta.inverse_SE3(T_world_ee) @ T_world_obstacle
dist = T_ee_obstacle[:3, 3]
controller.add_task(
    expr=dist,
    target= np.array([0, 0, 0.15]),
    gain_k=10,
    weight=1e4
)






##################################


#############################
# initialize the controller #
#############################
controller.initialize()

simulate(controller, q0)


qpOASES -- An Implementation of the Online Active Set Strategy.
Copyright (C) 2007-2015 by Hans Joachim Ferreau, Andreas Potschka,
Christian Kirches et al. All rights reserved.

qpOASES is distributed under the terms of the 
GNU Lesser General Public License 2.1 in the hope that it will be 
useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
See the GNU Lesser General Public License for more details.


qpOASES -- An Implementation of the Online Active Set Strategy.
Copyright (C) 2007-2015 by Hans Joachim Ferreau, Andreas Potschka,
Christian Kirches et al. All rights reserved.

qpOASES is distributed under the terms of the 
GNU Lesser General Public License 2.1 in the hope that it will be 
useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
See the GNU Lesser General Public License for more details.



### Task 3.3 : (Optional) Explore the nullspace of the robot

- 3.3.1 Provide the following task specification
    - Keep the end-effector at a fixed pose (position and/or orientation)
    - Wiggle some of the joints.    
- 3.3.2 Add a constraint to fix one of the robot joints (turning it into a 6 DOF robot), and repeat the task.
- 3.3.3 Rotate around a fixed position that is 0.1 m along the z-axis of the end-effector.

In [ ]:
controller = ConstraintBasedController(num_joints=7, mu=1e-4, R=np.ones(7))

q = controller.q
t = controller.t

# get the relevant transformations
T_world_ee = T_world_ee_fn(q)


####################
# add joint limits #
####################

controller.add_inequality(
    expr=q,
    ub=robot.model.upperPositionLimit[:7],
    lb=robot.model.lowerPositionLimit[:7],
    gain_k=10,
    hard=True
)

############################
# stay at a fixed position #
############################

##################################
# TODO: define constraints


##################################


#############################
# initialize the controller #
#############################
controller.initialize()

simulate(controller, q0)